# Filter artist/genre for mid-level energy songs
### For the purpose of working to

### Imports and Auth:

In [1]:
#!pip install -q spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import numpy as np
import pandas as pd
import os
from math import ceil, floor
import seaborn as sns
import sys
import time
import copy
from scipy.stats import iqr
#from google.colab import files

In [2]:
## clustering
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.preprocessing import MinMaxScaler

In [4]:
cid ="2bd837c837174d50ab873c61acf24f68" 
secret = "033bb4a851d54615bf83aaf94239ee06"
username = "omkwppodx5qi1pph0cwma65zc"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private streaming user-modify-playback-state user-read-playback-state'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri="http://localhost/")
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

### Next Steps:
#### Structure:
- enter artist or genre = x
- get id info for x
- get main playlist for x
    - eg "this is [x]" playlist 
    - or "best of [x]" album
- get metrics for all tracks
- figure out what constitutes low/mid/high range for energy



#### Artist

In [ ]:
## artist: -----------
raw_name_list = ["benny goodman", "harry james", "count basie", "artie shaw", "lucky millinder", "duke ellington", "glenn miller"]
artist_name_list = []

for name in raw_name_list:
    search = sp.search(name, limit=1, type="artist")

    in_genres = search["artists"]["items"][0]["genres"]
    artist_name = search["artists"]["items"][0]["name"]
    artist_id = search["artists"]["items"][0]["id"]

    artist_name_list.append(artist_name)

In [ ]:
for name in artist_name_list:
    query = "essential " + name
    srch = sp.search(q=query, limit=1, type="album")
    n = len(srch["albums"]["items"])
    alb_names = [srch["albums"]["items"][i]["name"] for i in range(n)]
    print(alb_names)

['The Essential Benny Goodman']
['Essential Collection (Rerecorded)']
['The Essential Count Basie']
['The Essential Artie Shaw']
[]
['The Essential Duke Ellington']
['The Essential Glenn Miller']


#### Get main playlist and album for artist:

In [ ]:
## get playlists -------
track_names = []
track_ids = []
track_uris = []
for name in artist_name_list:
    query = "this is " + name
    #print(query)
    pl_search = sp.search(q=query, limit=10, type="playlist")

    n = len(pl_search["playlists"]["items"])

    pl_names = [pl_search["playlists"]["items"][i]["name"] for i in range(n)]
    pl_ids = [pl_search["playlists"]["items"][i]["id"] for i in range(n)]
    
    good_names = [pl for pl in pl_names if pl.lower().find(query.lower()) != -1]
    good_idx = [good_names.index(nm) for nm in good_names]
    good_ids = [id for id in pl_ids if pl_ids.index(id) in good_idx]
    print(good_names)

    ## get tracks ----------
    n = len(good_ids)
    for i in range(n):
        pl_tracks = sp.playlist_tracks(good_ids[i])
        n_tracks = len(pl_tracks["items"])
        t_names = [pl_tracks["items"][j]["track"]["name"] for j in range(n_tracks)]
        t_ids = [pl_tracks["items"][j]["track"]["id"] for j in range(n_tracks)]
        t_uris = [pl_tracks["items"][j]["track"]["uri"] for j in range(n_tracks)]
        track_names.append(t_names)
        track_ids.append(t_ids)
        track_uris.append(t_uris)

## flatten lists
track_names = [item for sublist in track_names for item in sublist]
track_ids = [item for sublist in track_ids for item in sublist]
track_uris = [item for sublist in track_uris for item in sublist]

tracks_df = pd.DataFrame({
    "names" : track_names,
    "ids" : track_ids,
    "uris" : track_uris
})

['This Is Benny Goodman', 'This Is Benny Goodman Quartet', 'This Is Benny Goodman Trio', 'This Is Benny Goodman Sextet']
['This Is Harry James', 'This Is Harry James Orchestra']
['This Is Count Basie', 'This Is Count Basie Orchestra']
['This Is Artie Shaw', 'This Is Artie Shaw Orchestra']
['This Is Lucky Millinder']
['This Is Duke Ellington', 'This Is Duke Ellington Orchestra']
['This Is Glenn Miller', 'This Is Glenn Miller Orchestra']


In [ ]:
pl_tracks["items"][0]["track"]["uri"]

'spotify:track:0PJMHv4glk31FYeGq8Hg13'

### Create attributes df

In [ ]:
## get attributes and analyses of tracks
n_tracks = len(tracks_df)
track_50s = floor(n_tracks/50)
leftover = n_tracks % 50

if n_tracks/50 > 1:
    track_features = []
    for i in range(track_50s):
        slicer = slice(i*50, (i+1)*50)
        features = sp.audio_features(track_ids[slicer])
        track_features.append(features)
    ## get remainder
    slicer = slice(n_tracks-leftover, n_tracks)
    features = sp.audio_features(track_ids[slicer])
    track_features.append(features)
    track_features = [item for sublist in track_features for item in sublist]
else:
    track_features = sp.audio_features(track_ids)

track_analyses = []
for j in range(n_tracks):
    if j % 20 == 0:
        print(j)
    analysis = sp.audio_analysis(track_ids[j])
    track_analyses.append(analysis)


0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680


In [ ]:
len(track_features)

696

In [ ]:
## get attributes
ls = list(track_features[0].keys())
del ls [11:16]

for i in range(len(ls)):
    tracks_df[ls[i]] = [track_features[j][ls[i]] for j in range(len(track_features))]
tracks_df.columns

Index(['names', 'ids', 'uris', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [ ]:
track_analyses[10]["track"].keys()

dict_keys(['num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])

In [ ]:
## get audio analysis of tracks:
seg_lens = [len(track_analyses[i]["segments"]) for i in range(len(track_analyses))]

timbre_mean = []
timbre_max = []
timbre_min = []
timbre_sd = []
timbre_med = []
timbre_iqr = []
for tr in range(len(track_analyses)):
    timbre_ls = [track_analyses[tr]["segments"][i]["timbre"] for i in range(len(track_analyses[tr]["segments"]))]
    ## mean
    t_mean = np.mean(timbre_ls)
    timbre_mean.append(t_mean)
    ## max
    t_max = np.max(timbre_ls)
    timbre_max.append(t_max)
    ## min
    t_min = np.min(timbre_ls)
    timbre_min.append(t_min)
    ## std
    t_sd = np.std(timbre_ls)
    timbre_sd.append(t_sd)
    ## median
    t_med = np.median(timbre_ls)
    timbre_med.append(t_med)
    ## iqr
    t_q = iqr(timbre_ls)
    timbre_iqr.append(t_q)



In [ ]:
stats = ["mean", "max", "min", "sd", "med", "iqr"]
for stat in stats:
    col_name = "timbre_" + stat
    tracks_df[col_name] =  eval(col_name)



In [ ]:
tracks_df.drop_duplicates(inplace=True)
tracks_df = tracks_df[tracks_df.instrumentalness > 0.5]
tracks_df.head()

,names,ids,uris,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,timbre_mean,timbre_max,timbre_min,timbre_sd,timbre_med,timbre_iqr
1,"Sing, Sing, Sing",5L8ta4ECl5zeA6bGqY7G38,spotify:track:5L8ta4ECl5zeA6bGqY7G38,0.620,0.670,2,-11.575,0,0.0591,0.853,0.896,0.1560,0.295,113.492,520293,4,-2.095489,221.425,-208.807,41.528420,0.0100,35.55650
2,Chicago,5TjO0GOOon3R6YOnRFPwzk,spotify:track:5TjO0GOOon3R6YOnRFPwzk,0.595,0.285,8,-11.216,1,0.0501,0.943,0.729,0.1640,0.778,160.532,179840,3,6.867174,412.345,-339.171,46.421048,5.9875,47.94400
4,Goodbye,33epO6drRAdk6jWlwBkqC2,spotify:track:33epO6drRAdk6jWlwBkqC2,0.248,0.189,3,-13.594,0,0.0331,0.977,0.815,0.1250,0.142,75.662,198400,4,2.522711,179.716,-233.440,39.492342,-0.3735,40.83200
5,Don't Be That Way,7Bto5shUBaNnD3YDv1rnXv,spotify:track:7Bto5shUBaNnD3YDv1rnXv,0.689,0.267,7,-11.567,0,0.0728,0.870,0.866,0.0766,0.725,162.677,201587,4,10.597970,257.123,-316.683,44.309094,5.4625,47.86800
7,Christopher Columbus,2gXKmlCm7KIY5I8naTkbdU,spotify:track:2gXKmlCm7KIY5I8naTkbdU,0.621,0.219,10,-13.433,1,0.0749,0.740,0.921,0.1140,0.959,163.829,212213,4,11.608764,342.997,-238.922,45.558567,5.2225,49.52475


#### Initial Analysis:

In [ ]:
tracks_df.shape

(274, 21)

In [ ]:
## correlation plot
df = tracks_df.drop(columns=["names", "ids", "uris"])
#sns.pairplot(df, vars=["danceability", "energy", "valence", "tempo"])
sns.pairplot(df, vars=["speechiness", "energy", "instrumentalness", "tempo", "timbre_mean", "timbre_sd", "timbre_min", "timbre_max"])
#sns.pairplot(df, vars=["timbre_mean", "timbre_max", "timbre_min", "timbre_sd", "timbre_iqr", "timbre_med"])

#### filtering

In [ ]:
b = [4, 5, 6]
a = [1, 2, 3] + b

a

[1, 2, 3, 4, 5, 6]

In [ ]:
tracks_df[tracks_df.liveness > 0.7]

## Clustering

In [ ]:
tracks_df.columns

Index(['names', 'ids', 'uris', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'timbre_mean',
       'timbre_max', 'timbre_min', 'timbre_sd', 'timbre_med', 'timbre_iqr'],
      dtype='object')

In [ ]:
rmv_features = ["speechiness", "instrumentalness", "mode", "loudness", "acousticness", "liveness"]
X = tracks_df.drop(columns=["names", "ids", "uris"]+rmv_features).values
X = np.array(X)
print(tracks_df.shape, X.shape)

(274, 22) (274, 13)


In [ ]:
X_scaled = MinMaxScaler().fit_transform(X)

In [ ]:
cl = KMeans(n_clusters=3)

In [ ]:
cluster_labels = cl.fit_predict(X_scaled)

In [ ]:
tracks_df.names

1                   Sing, Sing, Sing
2                            Chicago
4                            Goodbye
5                  Don't Be That Way
7               Christopher Columbus
                   ...              
679           What A Wonderful World
686       Careless - Remastered 1996
687                  Memories Of You
689            Call Me Irresponsible
691    Tschaikowsky's Piano Concerto
Name: names, Length: 274, dtype: object

In [ ]:
cluster_df = tracks_df.copy()
cluster_df["label"] = cluster_labels

In [ ]:
cluster_df.groupby(by="label")["energy"].mean()

label
0    0.401548
1    0.183499
2    0.391134
Name: energy, dtype: float64

In [ ]:
cluster_df[cluster_df.label == 0]

,names,ids,uris,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,timbre_mean,timbre_max,timbre_min,timbre_sd,timbre_med,timbre_iqr,label
2,Chicago,5TjO0GOOon3R6YOnRFPwzk,spotify:track:5TjO0GOOon3R6YOnRFPwzk,0.595,0.285,8,-11.216,1,0.0501,0.943,0.729,0.1640,0.778,160.532,179840,3,6.867174,412.345,-339.171,46.421048,5.9875,47.94400,0
5,Don't Be That Way,7Bto5shUBaNnD3YDv1rnXv,spotify:track:7Bto5shUBaNnD3YDv1rnXv,0.689,0.267,7,-11.567,0,0.0728,0.870,0.866,0.0766,0.725,162.677,201587,4,10.597970,257.123,-316.683,44.309094,5.4625,47.86800,0
7,Christopher Columbus,2gXKmlCm7KIY5I8naTkbdU,spotify:track:2gXKmlCm7KIY5I8naTkbdU,0.621,0.219,10,-13.433,1,0.0749,0.740,0.921,0.1140,0.959,163.829,212213,4,11.608764,342.997,-238.922,45.558567,5.2225,49.52475,0
14,Let's Dance,6DjOS2PYRPQ8amyhSpdErX,spotify:track:6DjOS2PYRPQ8amyhSpdErX,0.517,0.431,8,-9.537,1,0.0551,0.980,0.856,0.0945,0.892,203.050,152960,4,7.649696,300.992,-380.536,38.908368,0.7000,49.02450,0
20,"Oh, Lady Be Good",1BVXsXFvHHNHnJ4KRFaP6R,spotify:track:1BVXsXFvHHNHnJ4KRFaP6R,0.614,0.306,7,-14.475,1,0.0432,0.943,0.753,0.1340,0.942,176.607,182333,4,10.840481,302.959,-284.139,44.277918,4.2220,50.36725,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,King Porter Stomp,1ZOj7lwZ6qMPr66CYEw3U4,spotify:track:1ZOj7lwZ6qMPr66CYEw3U4,0.583,0.486,8,-11.412,1,0.0961,0.760,0.630,0.0907,0.759,187.483,213360,4,10.753732,286.033,-373.579,42.044991,5.7075,40.41025,0
646,In The Mood,0PJMHv4glk31FYeGq8Hg13,spotify:track:0PJMHv4glk31FYeGq8Hg13,0.613,0.394,8,-12.934,1,0.0651,0.661,0.763,0.0620,0.719,164.745,220707,4,13.577284,301.975,-174.445,37.794957,7.3830,44.39450,0
660,The Jumpin' Jive,1fb1mlR1oVlMt1Z3eeUciv,spotify:track:1fb1mlR1oVlMt1Z3eeUciv,0.514,0.592,9,-12.926,0,0.0522,0.947,0.623,0.0895,0.812,193.977,153160,4,8.529574,281.233,-247.785,35.304159,4.4565,46.60700,0
667,Bugle Call Rag,3QoVVKQCvq5chlJD40v7Ej,spotify:track:3QoVVKQCvq5chlJD40v7Ej,0.439,0.593,10,-10.012,1,0.0509,0.812,0.680,0.0805,0.806,122.209,155293,4,8.861539,207.365,-164.732,36.035983,5.9925,44.77875,0


## playback for testing

In [12]:
cluster_df = pd.read_csv("testing_df.csv")

In [13]:
uris = cluster_df[cluster_df.label == 0]["uris"].values

In [14]:
sp.start_playback(uris=list(uris))

In [ ]:
sp.next_track()

In [ ]:
current_track = sp.current_playback()["item"]

In [ ]:
print(current_track["name"], current_track["uri"], current_track["id"])

Runnin' Wild spotify:track:3f5gk0zaG14vEUkr3EymUI 3f5gk0zaG14vEUkr3EymUI


In [ ]:
testing_df = cluster_df.copy()
testing_df["good_cluster"] = ""
testing_df["energy_level"] = ""

In [ ]:
track_uri = "spotify:track:7Bto5shUBaNnD3YDv1rnXv"
row_index = testing_df.index[testing_df.uris == track_uri]
testing_df.at[row_index.values[0], "names"]


"Don't Be That Way"

In [ ]:
def label_track(energy, good_cluster=0):
    current_track = sp.current_playback()["item"]
    print(current_track["name"])
    track_id = current_track["id"]
    track_uri = current_track["uri"]
    row_index = testing_df.index[testing_df.uris == track_uri].values[0]
    testing_df.at[row_index, "good_cluster"] = good_cluster
    testing_df.at[row_index, "energy_level"] = energy

In [ ]:
current_track = sp.current_playback()["item"]
current_track["name"]

'Carnival'

In [ ]:
c = 1

In [ ]:
e = 1

In [ ]:
label_track(energy=e, good_cluster=c)

Carnival


In [ ]:
testing_df.at[testing_df.index[testing_df.uris == current_track["uri"]].values[0], "energy_level"]

1

In [ ]:
sp.next_track()

#### write out data

In [ ]:
testing_df.to_csv("testing_df.csv")
files.download('testing_df.csv')